In [21]:
import os
import logging
from datetime import datetime
import json

from csbdeep.models import Config
import onnxruntime as ort
import onnx

from flame import FLAMEImage

In [4]:
assert 'CUDAExecutionProvider' in ort.get_available_providers()

In [26]:
DATASET_DIRECTORY = "/mnt/d/code/Balu_CARE/datasets"
DATASET_NAME = "20250513_40I_denoising_7to40F"
DATASET_JSON = os.path.join(DATASET_DIRECTORY, f"{DATASET_NAME}.json")
MODEL_DIRECTORY = "/mnt/d/models/CARE/test_model"
MODEL_NAME = os.path.basename(MODEL_DIRECTORY)
ONNX_PATH = os.path.join(MODEL_DIRECTORY, f"{MODEL_NAME}.onnx")
JSON_PATH = os.path.join(MODEL_DIRECTORY, f"config.json")

for f in [DATASET_JSON, ONNX_PATH, JSON_PATH]:
    assert os.path.isfile(f)

### Loading Images

In [27]:
model_config_dict = json.load(open(JSON_PATH, 'r'))
dataset_config_dict = json.load(open(DATASET_JSON, 'r'))

In [28]:
dataset_config_dict['FLAME_Dataset']

{'name': '20250513_40I_denoising_7to40F',
 'type': 'denoising',
 'input': {'n_frames': 7,
  'pixel_mean': 482.47699672453706,
  'pixel_min': 0,
  'pixel_max': 15400,
  'pixel_p1pct': [0, 0, 0],
  'pixel_1pct': [0, 0, 0],
  'pixel_5pct': [0, 0, 0],
  'pixel_95pct': [800, 1000, 1875],
  'pixel_99pct': [1165, 1424, 2831],
  'pixel_99p9pct': [3164, 4250, 4664],
  'pixel_std': 515.7695874822199},
 'output': {'frames': 40,
  'pixel_mean': 2756.5133061342594,
  'pixel_min': 0,
  'pixel_max': 81200,
  'pixel_p1pct': [0, 0, 0],
  'pixel_1pct': [0, 0, 400],
  'pixel_5pct': [200, 284, 1000],
  'pixel_95pct': [3200, 4400, 9547],
  'pixel_99pct': [4653, 6405, 15200],
  'pixel_99p9pct': [17400, 24200, 25488],
  'pixel_std': 2561.295601730073},
 'image_ids': [11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80]}

### Loading ONNX

In [8]:
onnx_model = onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

In [10]:
ort_session = ort.InferenceSession(
    ONNX_PATH,
    providers=['CUDAExecutionProvider']
)

2025-05-15 11:31:27.123088841 [W:onnxruntime:, session_state.cc:1280 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-05-15 11:31:27.123117107 [W:onnxruntime:, session_state.cc:1282 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [19]:
input_tensor = ort_session.get_inputs()[0]
input_name, input_shape, input_type = input_tensor.name, input_tensor.shape, input_tensor.type
print(f"Input Tensor\nName: {input_name}\nShape: {input_shape}\nType: {input_type}")

Input Tensor
Name: patch
Shape: ['unk__193', 128, 128, 3]
Type: tensor(float)
